In [1]:
# Imports

import tensorflow as tf
import tensorflow_datasets as tfds
#import tensorflowjs
import numpy as np
import os
import PIL.Image
from PIL import ImageOps
#from PIL import Image
import PIL
import pathlib
import matplotlib.pyplot as plt
import datetime
import tensorboard
import IPython
import sklearn
import cv2
import subprocess
import sys

2022-06-21 12:28:57.587306: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-21 12:28:57.587355: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
# Dataset loading

data_path = pathlib.Path('/home/samer/Documents/Programming/AI50xIraq/CancerDetectionX3/archiveX3/Training/')
#data_dir = pathlib.Path(data_path)
data_path_test = pathlib.Path('/home/samer/Documents/Programming/AI50xIraq/CancerDetectionX3/archiveX3/Testing/')

dataset_path = tf.keras.utils.image_dataset_from_directory(
    data_path,
    labels= 'inferred',
    validation_split=0.2,
    subset='training',
    seed= 1,
    batch_size=5,
    image_size=(180, 180),
    color_mode="rgb",
    shuffle=True)

dataset_path_val = tf.keras.utils.image_dataset_from_directory(
    data_path,
    labels= 'inferred',
    validation_split=0.2,
    subset='validation',
    seed= 2,
    batch_size=5,
    image_size=(180, 180),
    color_mode="rgb",
    shuffle=True)

dataset_path_test = tf.keras.utils.image_dataset_from_directory(
    data_path_test,
    labels= 'inferred',
    seed= 3,
    batch_size=5,
    image_size=(180, 180),
    color_mode="rgb",
    shuffle=True)


In [ ]:
# catch and prefetch
AUTOTUNE=tf.data.AUTOTUNE
dataset_path = dataset_path.cache().prefetch(buffer_size=AUTOTUNE)
dataset_path_val = dataset_path_val.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# downloading the weights

# base_model = tf.keras.applications.inception_v3.InceptionV3(
#     input_shape = (180, 180, 3),
#     include_top = False, 
#     weights = "imagenet"                   # does not work for medical imaging
# )

In [ ]:
#saving the model

# saving_path = pathlib.Path('/home/samer/Documents/Programming/AI50xIraq/CancerDetectionX3/SavedBaseModel.h5')

# base_model.save(saving_path)

In [ ]:
model_path = pathlib.Path('/home/samer/Documents/Programming/AI50xIraq/CancerDetectionX3/SavedBaseModel.h5')

base_model = tf.keras.models.load_model(model_path)

base_model.trainable = False

rescaling = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255)
])
augmentation = tf.keras.Sequential([
   tf.keras.layers.RandomFlip("horizontal_and_vertical"),
   tf.keras.layers.RandomRotation(0.2)
])

inputs = tf.keras.Input(shape=(180, 180, 3))
x = augmentation(inputs)
x = rescaling(x)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(256, activation='ELU', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(256, activation='ELU', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(4, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)


model.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits= True),
    metrics=['accuracy','mse'])

model.fit(
    dataset_path,
    epochs=15,                                    #get it to 15 from 30
    validation_data = dataset_path_val
    #callbacks =[tensorboard_callback,
                 #cp_callback
                 )